In [1]:
% matplotlib inline
% load_ext autoreload
% autoreload 2

In [2]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from deep_visual_odometry.kitti_utils import KITTIdata
from matplotlib import pyplot as plt

In [3]:
basedir = 'dataset'

In [4]:
from deep_visual_odometry.odometry_model import OdomModel

In [5]:
batch_size = 256         # Sequences per batch
num_steps = 8         # Number of sequence steps per batch
rnn_size = 64          # Size of hidden layers in rnn_cell
num_layers = 2           # Number of hidden layers
learning_rate = 0.001

In [6]:
kitti_data = KITTIdata(basedir, 
                       sequences = ['01'],
                       sequence_len = num_steps,
                       val_frac = 0,
                       test_frac = 0.1,
                       img_size=(150,50))

completed load sequence 01 data


In [7]:
velo = []
pose = []
for sequence in kitti_data.sequences:
    velo.append(kitti_data.velocities[sequence])
    pose.append(kitti_data.poses[sequence])
velo = np.vstack(velo)
pose = np.vstack(pose)

mean_velo = np.mean(velo,axis = 0)
std_velo = np.std(velo,axis = 0)
mean_pose = np.mean(pose,axis = 0)
std_pose = np.std(pose,axis = 0)


In [8]:
for sequence in kitti_data.sequences:
    kitti_data.velocities[sequence] = kitti_data.velocities[sequence] - mean_velo
    kitti_data.velocities[sequence] /= std_velo
    
    kitti_data.poses[sequence] = kitti_data.poses[sequence] - mean_pose
    kitti_data.poses[sequence] /= std_pose

In [10]:
model = OdomModel(batch_size = 256, 
                  num_steps = 8,
                  cell_type = 'GRU',
                  rnn_size = 128,
                  num_layers = 2,
                  learning_rate = 0.001)
model.train(kitti_data, max_count = 500, save_every_n = 1000)

output size (256, 8, 128)
seq_output shape (256, 8, 128)
shape of outputs (256, 8, 3)
shape of targets (256, 8, 3)
step: 500  loss: 0.4579  0.0809 sec/batch


In [12]:
inputs_test, velocities_test, poses_test = kitti_data.load_data_test()

In [15]:
velocities_test.shape

(109, 8, 2)

In [22]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
predictions = model.test(checkpoint,velocities_test, 256)

INFO:tensorflow:Restoring parameters from checkpoints/i501_l128.ckpt


In [20]:
predictions.shape

(0, 8, 3)

In [ ]:
poses_test[0:512].shape

In [ ]:
i = 2
j = 2
plt.plot(predictions[i,:, j],label = 'pred')
plt.plot(poses_test[i,:, j],label = 'ground truth')
plt.legend()
plt.xlabel('sequence num')
plt.ylabel('theta')
#plt.savefig('theta_vs_seq.jpg')
